# Physics-Informed Neural Networks (PINN) — Универсальный шаблон проекта

Этот ноутбук демонстрирует структуру и реализацию проекта **PINN (Physics-Informed Neural Network)**.

**PINN** — это метод, который использует нейронные сети для решения дифференциальных уравнений, включая физические ограничения прямо в функцию потерь.

Проект был разработан в парадигме ООП на языке Python 3, таким образом возможны дальнейшие модификации путем изменения небольшого числа строк.

## Исследуемая задача

Задача 5. Исследование влияния весового коэффициента в функции потерь PINN на точность решения.

Двумерное уравнение Пуассона:

$$
\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = f(x, y), \quad (x, y) \in \Omega.
$$

Граничные условия Дирихле:

$$
u(x, y) = g(x, y), \quad (x, y) \in \partial \Omega.
$$

**Задача:** Реализовать PINN метод решения краевой задачи для уравнения Пуассона. Исследовать влияние весового коэффициента $\lambda$ в функции потерь PINN на точность и эффективность решения.

Для простоты исследования были выбраны такие функции $f(x, y)$ и $g(x, y)$, что дифференциальное уравнение имеет аналитическое решение. Так, например, подходят функции $f(x, y) = -2 \pi^2 \sin(\pi x) \sin(\pi y)$ и $g(x, y) \equiv 0$. Тогда решение приобретает следующий вид: $u(x, y) = \sin(\pi x) \sin(\pi y)$

## 1. Импорт библиотек

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import random, os
import matplotlib.pyplot as plt
from dataclasses import dataclass, field, replace
from abc import ABC, abstractmethod
from datetime import datetime

## 2. Конфигурация

In [ ]:
@dataclass(frozen=True)
class Config:
    seed: int = 69
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    epochs: int = 5000
    lr: float = 1e-4
    N_f: int = 5000
    N_b: int = 1000
    layers: tuple = (64, 64, 64)
    lam_pde: float = 1.0
    lam_bc: float = 1e3
    verbose_every: int = 1000

cfg = Config()
print(cfg)

## 3. Генератор данных

In [ ]:
class DataGenerator:
    @staticmethod
    def domain_points(N_f, device='cpu'):
        x = torch.rand(N_f, 1, device=device)
        y = torch.rand(N_f, 1, device=device)
        return x, y

    @staticmethod
    def boundary_points(N_b, device='cpu'):
        t = torch.linspace(0, 1, N_b, device=device).view(-1, 1)
        x_b = torch.cat([torch.zeros_like(t), torch.ones_like(t), t, t], dim=0)
        y_b = torch.cat([t, t, torch.zeros_like(t), torch.ones_like(t)], dim=0)
        return x_b, y_b

## 4. Определение PDE

In [ ]:
def u_exact(x, y):
    return torch.sin(torch.pi * x) * torch.sin(torch.pi * y)

def f_func(x, y):
    return -2 * (torch.pi**2) * torch.sin(torch.pi * x) * torch.sin(torch.pi * y)

def g_func(x, y):
    return torch.zeros_like(x)

## 5. Модель PINN

In [ ]:
class PINN(nn.Module):
    def __init__(self, input_size=2, hidden_sizes=(64,64,64), output_size=1, activation=nn.Tanh()):
        super().__init__()
        layers = [nn.Linear(input_size, hidden_sizes[0]), activation]
        for i in range(len(hidden_sizes)-1):
            layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
            layers.append(activation)
        layers.append(nn.Linear(hidden_sizes[-1], output_size))
        self.net = nn.Sequential(*layers)

    def forward(self, xy):
        return self.net(xy)

## 6. Функции потерь

In [ ]:
class Losses:
    @staticmethod
    def pde_loss(model, xy, f_func):
        xy.requires_grad_(True)
        u = model(xy)
        grads = torch.autograd.grad(u, xy, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_x, u_y = grads[:,0:1], grads[:,1:2]
        u_xx = torch.autograd.grad(u_x, xy, grad_outputs=torch.ones_like(u_x), create_graph=True)[0][:,0:1]
        u_yy = torch.autograd.grad(u_y, xy, grad_outputs=torch.ones_like(u_y), create_graph=True)[0][:,1:2]
        residual = u_xx + u_yy - f_func(xy[:,0:1], xy[:,1:2])
        return torch.mean(residual**2)

    @staticmethod
    def boundary_loss(model, x_b, y_b, g_func):
        xy_b = torch.cat([x_b, y_b], dim=1)
        residual = model(xy_b) - g_func(x_b, y_b)
        return torch.mean(residual**2)

## 7. Тренировочный цикл

In [ ]:
@dataclass
class TrainingHistory:
    total_loss: list = field(default_factory=list)
    pde_loss: list = field(default_factory=list)
    bc_loss: list = field(default_factory=list)

class Trainer:
    def __init__(self, model, config, f_func, g_func, data_gen):
        self.model = model
        self.cfg = config
        self.f_func = f_func
        self.g_func = g_func
        self.data_gen = data_gen
        self.history = TrainingHistory()

    def train(self):
        opt = torch.optim.Adam(self.model.parameters(), lr=self.cfg.lr)
        x_b, y_b = self.data_gen.boundary_points(self.cfg.N_b, device=self.cfg.device)
        self.model.train()
        for epoch in range(self.cfg.epochs+1):
            opt.zero_grad()
            x_f, y_f = self.data_gen.domain_points(self.cfg.N_f, device=self.cfg.device)
            xy_f = torch.cat([x_f, y_f], dim=1)
            loss_pde = Losses.pde_loss(self.model, xy_f, self.f_func)
            loss_bc = Losses.boundary_loss(self.model, x_b, y_b, self.g_func)
            loss = self.cfg.lam_pde * loss_pde + self.cfg.lam_bc * loss_bc
            loss.backward(); opt.step()
            self.history.total_loss.append(loss.item())
            self.history.pde_loss.append(loss_pde.item())
            self.history.bc_loss.append(loss_bc.item())
            if epoch % self.cfg.verbose_every == 0:
                print(f"[{epoch}] Total={loss.item():.3e} PDE={loss_pde.item():.3e} BC={loss_bc.item():.3e}")
        return self.model, self.history

## 8. Пример запуска обучения

In [ ]:
cfg = Config(epochs=2000, verbose_every=500)
data_gen = DataGenerator()
model = PINN(hidden_sizes=cfg.layers).to(cfg.device)
trainer = Trainer(model, cfg, f_func, g_func, data_gen)
model, history = trainer.train()

## 9. Визуализация результата

In [ ]:
x = np.linspace(0,1,100)
y = np.linspace(0,1,100)
X, Y = np.meshgrid(x, y)
xy = torch.tensor(np.c_[X.ravel(), Y.ravel()], dtype=torch.float32)
with torch.no_grad():
    U_pred = model(xy).cpu().numpy().reshape(100,100)
U_true = np.sin(np.pi*X)*np.sin(np.pi*Y)
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.title('Prediction')
plt.contourf(X, Y, U_pred, 100, cmap='viridis')
plt.subplot(1,2,2)
plt.title('Error')
plt.contourf(X, Y, np.abs(U_pred-U_true), 100, cmap='hot')
plt.show()